this script takes as input the LSTM or RNN weights found by train.py
change the path in line 178 of this script to point to the h5 file
with LSTM or RNN weights generated by train.py

Author: Niek Tax

In [1]:
from __future__ import division
from keras.models import load_model
from theano.ifelse import ifelse #added this
import csv
import copy
import numpy as np
import distance
from itertools import izip, islice
from jellyfish._jellyfish import damerau_levenshtein_distance
import unicodecsv
from sklearn import metrics
from math import sqrt
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import Counter

Using Theano backend.


# Read 1

In [2]:
lastcase = ''
line = ''
firstLine = True
lines = []
timeseqs = []
timeseqs2 = []
times = []
times2 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [3]:
eventlog = "helpdesk.csv"
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
ascii_offset = 161

In [4]:
for row in islice(spamreader, 0, 174):
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:        
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
        line = ''
        times = []
        times2 = []
        numlines+=1
    line+=unichr(int(row[1])+ascii_offset)
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    times.append(timediff)
    times2.append(timediff2)
    lasteventtime = t
    firstLine = False

# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
numlines+=1

In [5]:
divisor = np.mean([item for sublist in timeseqs for item in sublist])
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist])
print('divisor2: {}'.format(divisor2))
divisor3 = np.mean(map(lambda x: np.mean(map(lambda y: x[len(x)-1]-y, x)), timeseqs2))
print('divisor3: {}'.format(divisor3))

divisor: 209169.482759
divisor2: 399919.678161
divisor3: 368261.365476


In [6]:
elems_per_fold = int(round(numlines/3))
fold1 = lines[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]

fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]

lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2

In [7]:
step = 1
sentences = []
softness = 0
next_chars = []
lines = map(lambda x: x+'!',lines)
maxlen = max(map(lambda x: len(x),lines))

chars = map(lambda x : set(x),lines)
chars = list(set().union(*chars))
chars.sort()
target_chars = copy.copy(chars)
chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

total chars: 5, target chars: 6
{0: u'\xa2', 1: u'\xa4', 2: u'\xa7', 3: u'\xa9', 4: u'\xaa'}


In [8]:
maxlen

9

# Read 2

In [9]:
lastcase = ''
line = ''
firstLine = True
lines = []
timeseqs = []  # relative time since previous event
timeseqs2 = [] # relative time since case start
timeseqs3 = [] # absolute time of previous event
times = []
times2 = []
times3 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [10]:
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers

['CaseID', 'ActivityID', 'CompleteTimestamp']

In [11]:
for row in islice(spamreader, 0, 174):
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:        
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
        line = ''
        times = []
        times2 = []
        times3 = []
        numlines+=1
    line+=unichr(int(row[1])+ascii_offset)
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    times.append(timediff)
    times2.append(timediff2)
    times3.append(datetime.fromtimestamp(time.mktime(t)))
    lasteventtime = t
    firstLine = False

# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
numlines+=1

In [12]:
lines[:5]

[u'\xa2\xa9\xa7',
 u'\xa2\xa9\xa7',
 u'\xa2\xa9\xa7',
 u'\xa2\xa9\xa7\xa9\xa7',
 u'\xa2\xa9\xa7']

In [13]:
timeseqs[:5]

[[0, 15, 173999],
 [0, 457325, 306],
 [0, 62954, 432],
 [0, 2221, 109, 1660, 6746],
 [0, 2697, 64551]]

In [14]:
timeseqs2[:5]

[[0, 15, 174014],
 [0, 457325, 457631],
 [0, 62954, 63386],
 [0, 2221, 2330, 3990, 10736],
 [0, 2697, 67248]]

In [15]:
timeseqs3[:5]

[[datetime.datetime(2012, 4, 3, 16, 55, 38),
  datetime.datetime(2012, 4, 3, 16, 55, 53),
  datetime.datetime(2012, 4, 5, 17, 15, 52)],
 [datetime.datetime(2010, 10, 29, 18, 14, 6),
  datetime.datetime(2010, 11, 4, 1, 16, 11),
  datetime.datetime(2010, 11, 4, 1, 21, 17)],
 [datetime.datetime(2010, 12, 15, 23, 31, 53),
  datetime.datetime(2010, 12, 16, 17, 1, 7),
  datetime.datetime(2010, 12, 16, 17, 8, 19)],
 [datetime.datetime(2012, 4, 3, 21, 8, 32),
  datetime.datetime(2012, 4, 3, 21, 45, 33),
  datetime.datetime(2012, 4, 3, 21, 47, 22),
  datetime.datetime(2012, 4, 3, 22, 15, 2),
  datetime.datetime(2012, 4, 4, 0, 7, 28)],
 [datetime.datetime(2011, 10, 25, 23, 13, 58),
  datetime.datetime(2011, 10, 25, 23, 58, 55),
  datetime.datetime(2011, 10, 26, 17, 54, 46)]]

In [16]:
numlines, len(lines), len(timeseqs), len(timeseqs2), len(timeseqs3)

(51, 50, 50, 50, 50)

In [17]:
fold3 = lines[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]

lines = fold3
lines_t = fold3_t
lines_t2 = fold3_t2
lines_t3 = fold3_t3

In [18]:
elems_per_fold, len(lines), len(lines_t), len(lines_t2), len(lines_t3)

(17, 16, 16, 16, 16)

In [19]:
lines[:5]

[u'\xa2\xa9\xaa\xa7',
 u'\xa2\xa9\xa7\xa7',
 u'\xa2\xa9\xaa\xa7',
 u'\xa2\xa9\xa7',
 u'\xa2\xa9\xaa\xaa\xa9\xa7']

In [20]:
lines_t[:5]

[[0, 834, 345587, 265102],
 [0, 410, 20283, 1],
 [0, 61890, 86961, 1477477],
 [0, 9, 70073],
 [0, 83363, 1131019, 0, 80946, 504371]]

In [21]:
lines_t2[:5]

[[0, 834, 346421, 611523],
 [0, 410, 20693, 20694],
 [0, 61890, 148851, 1626328],
 [0, 9, 70082],
 [0, 83363, 1214382, 1214382, 1295328, 1799699]]

In [22]:
lines_t3[:5]

[[datetime.datetime(2011, 11, 25, 23, 33, 54),
  datetime.datetime(2011, 11, 25, 23, 47, 48),
  datetime.datetime(2011, 11, 29, 23, 47, 35),
  datetime.datetime(2011, 12, 3, 1, 25, 57)],
 [datetime.datetime(2012, 1, 16, 19, 53, 1),
  datetime.datetime(2012, 1, 16, 19, 59, 51),
  datetime.datetime(2012, 1, 17, 1, 37, 54),
  datetime.datetime(2012, 1, 17, 1, 37, 55)],
 [datetime.datetime(2012, 6, 13, 21, 55, 36),
  datetime.datetime(2012, 6, 14, 15, 7, 6),
  datetime.datetime(2012, 6, 15, 15, 16, 27),
  datetime.datetime(2012, 7, 2, 17, 41, 4)],
 [datetime.datetime(2012, 7, 26, 23, 18, 26),
  datetime.datetime(2012, 7, 26, 23, 18, 35),
  datetime.datetime(2012, 7, 27, 18, 46, 28)],
 [datetime.datetime(2010, 10, 14, 21, 53, 13),
  datetime.datetime(2010, 10, 15, 21, 2, 36),
  datetime.datetime(2010, 10, 28, 23, 12, 55),
  datetime.datetime(2010, 10, 28, 23, 12, 55),
  datetime.datetime(2010, 10, 29, 21, 42, 1),
  datetime.datetime(2010, 11, 4, 17, 48, 12)]]

In [23]:
# set parameters
predict_size = maxlen

# load model, set this to the model generated by train.py
model = load_model('output_files/models/model_218-0.37.h5')

/Users/hoangnguyen/miniconda3/envs/pydata2/lib/python2.7/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


# Helper function 

In [24]:
# define helper functions
def encode(sentence, times, times3, maxlen=maxlen):
    num_features = len(chars)+5
    X = np.zeros((1, maxlen, num_features), dtype=np.float32)
    leftpad = maxlen-len(sentence)
    times2 = np.cumsum(times)
    for t, char in enumerate(sentence):
        midnight = times3[t].replace(hour=0, minute=0, second=0, microsecond=0)
        timesincemidnight = times3[t]-midnight
        multiset_abstraction = Counter(sentence[:t+1])
        for c in chars:
            if c==char:
                X[0, t+leftpad, char_indices[c]] = 1
        X[0, t+leftpad, len(chars)] = t+1
        X[0, t+leftpad, len(chars)+1] = times[t]/divisor
        X[0, t+leftpad, len(chars)+2] = times2[t]/divisor2
        X[0, t+leftpad, len(chars)+3] = timesincemidnight.seconds/86400
        X[0, t+leftpad, len(chars)+4] = times3[t].weekday()/7
    return X

In [25]:
def getSymbol(predictions):
    maxPrediction = 0
    symbol = ''
    i = 0;
    for prediction in predictions:
        if(prediction>=maxPrediction):
            maxPrediction = prediction
            symbol = target_indices_char[i]
        i += 1
    return symbol

# Predict

In [26]:
one_ahead_gt = []
one_ahead_pred = []

two_ahead_gt = []
two_ahead_pred = []

three_ahead_gt = []
three_ahead_pred = []

In [27]:
# make predictions
with open('output_files/results/suffix_and_remaining_time_%s' % eventlog, 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Prefix length", "Groud truth", "Predicted", "Levenshtein", "Damerau", "Jaccard", "Ground truth times", "Predicted times", "RMSE", "MAE", "Median AE"])
    for prefix_size in range(2,maxlen):
        print(prefix_size)
        for line, times, times2, times3 in izip(lines, lines_t, lines_t2, lines_t3):
            times.append(0)
            cropped_line = ''.join(line[:prefix_size])
            cropped_times = times[:prefix_size]
            cropped_times3 = times3[:prefix_size]
            if len(times2)<prefix_size:
                continue # make no prediction for this case, since this case has ended already
            ground_truth = ''.join(line[prefix_size:prefix_size+predict_size])
            ground_truth_t = times2[prefix_size-1]
            case_end_time = times2[len(times2)-1]
            ground_truth_t = case_end_time-ground_truth_t
            
            predicted = ''
            total_predicted_time = 0
            for i in range(predict_size):
                enc = encode(cropped_line, cropped_times, cropped_times3) # encode the input
                y = model.predict(enc, verbose=0) # make predictions
                
                # split predictions into seperate activity and time predictions
                y_char = y[0][0] 
                y_t = y[1][0][0]
                prediction = getSymbol(y_char) # undo one-hot encoding           
                cropped_line += prediction
                
                if y_t<0:
                    y_t=0
                cropped_times.append(y_t)
                
                if prediction == '!': # end of case was just predicted, therefore, stop predicting further into the future
                    one_ahead_pred.append(total_predicted_time)
                    one_ahead_gt.append(ground_truth_t)
                    print('! predicted, end case')
                    break
                y_t = y_t * divisor3
                cropped_times3.append(cropped_times3[-1] + timedelta(seconds=y_t))
                total_predicted_time = total_predicted_time + y_t
                predicted += prediction
            output = []
            if len(ground_truth)>0:
                output.append(prefix_size)
                output.append(unicode(ground_truth).encode("utf-8"))
                output.append(unicode(predicted).encode("utf-8"))
                
                output.append(1 - distance.nlevenshtein(predicted, ground_truth))
                dls = 1 - (damerau_levenshtein_distance(unicode(predicted), unicode(ground_truth)) / max(len(predicted),len(ground_truth)))
                
                if dls<0:
                    dls=0 # we encountered problems with Damerau-Levenshtein Similarity on some linux machines where the default character encoding of the operating system caused it to be negative, this should never be the case
                output.append(dls)
                
                output.append(1 - distance.jaccard(predicted, ground_truth))
                
                output.append(ground_truth_t)
                output.append(total_predicted_time)
                output.append('')
                
                output.append(metrics.mean_absolute_error([ground_truth_t], [total_predicted_time]))
                output.append(metrics.median_absolute_error([ground_truth_t], [total_predicted_time]))
                
                spamwriter.writerow(output)

2


ValueError: Error when checking : expected main_input to have shape (None, 6, 8) but got array with shape (1, 9, 10)

In [ ]:
lines_t[0]

In [ ]:
for prefix_size in range(2,maxlen):
        print(prefix_size)
        for line, times, times2, times3 in izip(lines, lines_t, lines_t2, lines_t3):
            times.append(0)
            cropped_line = ''.join(line[:prefix_size])
            cropped_times = times[:prefix_size]
            cropped_times3 = times3[:prefix_size]
            if len(times2)<prefix_size:
                continue # make no prediction for this case, since this case has ended already
            ground_truth = ''.join(line[prefix_size:prefix_size+predict_size])
            ground_truth_t = times2[prefix_size-1]
            case_end_time = times2[len(times2)-1]
            ground_truth_t = case_end_time-ground_truth_t
            predicted = ''
            total_predicted_time = 0
            
            #print('line: {}'.format(line))
            print('times: {}'.format(times))
            print('times2: {}'.format(times2))
            print('times3: {}'.format(times3))
            #print('cropped_line: {}'.format(cropped_line))
            print('cropped_times: {}'.format(cropped_times))
            print('cropped_times3: {}'.format(cropped_times3))
            print('------------------------------------------')
            #print('ground_truth: {}'.format(ground_truth))
            print('ground_truth_t: {}'.format(ground_truth_t))
            print('case_end_time: {}'.format(case_end_time))
            print('\n')

In [28]:
prefix_size = 2
predict_size = 1
line = lines[0]
times = lines_t[0] 
times2 = lines_t2[0] 
times3 = lines_t3[0]

print('times: {}'.format(times))
print('times2: {}'.format(times2))
print('times3: {}'.format(times3))

#times.append(0)
cropped_line = ''.join(line[:prefix_size])
cropped_times = times[:prefix_size]
cropped_times3 = times3[:prefix_size]

#if len(times2)<prefix_size:
#    continue # make no prediction for this case, since this case has ended already

ground_truth = ''.join(line[prefix_size:prefix_size+predict_size])
ground_truth_t = times2[prefix_size-1]
print(ground_truth_t)
case_end_time = times2[len(times2)-1]
ground_truth_t = case_end_time-ground_truth_t #find
            
#print('line: {}'.format(line))
print('times: {}'.format(times))
print('times2: {}'.format(times2))
print('times3: {}'.format(times3))
#print('cropped_line: {}'.format(cropped_line))
print('cropped_times: {}'.format(cropped_times))
print('cropped_times3: {}'.format(cropped_times3))
print('------------------------------------------')
#print('ground_truth: {}'.format(ground_truth))
print('ground_truth_t: {}'.format(ground_truth_t))
print('Check: {}'.format(times2[prefix_size-1]))
print('case_end_time: {}'.format(case_end_time))
print('Check: {}'.format(times2[len(times2)-1]))
print('\n')

times: [0, 834, 345587, 265102, 0]
times2: [0, 834, 346421, 611523]
times3: [datetime.datetime(2011, 11, 25, 23, 33, 54), datetime.datetime(2011, 11, 25, 23, 47, 48), datetime.datetime(2011, 11, 29, 23, 47, 35), datetime.datetime(2011, 12, 3, 1, 25, 57)]
834
times: [0, 834, 345587, 265102, 0]
times2: [0, 834, 346421, 611523]
times3: [datetime.datetime(2011, 11, 25, 23, 33, 54), datetime.datetime(2011, 11, 25, 23, 47, 48), datetime.datetime(2011, 11, 29, 23, 47, 35), datetime.datetime(2011, 12, 3, 1, 25, 57)]
cropped_times: [0, 834]
cropped_times3: [datetime.datetime(2011, 11, 25, 23, 33, 54), datetime.datetime(2011, 11, 25, 23, 47, 48)]
------------------------------------------
ground_truth_t: 610689
Check: 834
case_end_time: 611523
Check: 611523




In [29]:
line[:2]

u'\xa2\xa9'

In [30]:
cropped_line, cropped_times, cropped_times3

(u'\xa2\xa9',
 [0, 834],
 [datetime.datetime(2011, 11, 25, 23, 33, 54),
  datetime.datetime(2011, 11, 25, 23, 47, 48)])

In [31]:
enc = encode(cropped_line, cropped_times, cropped_times3)

In [32]:
enc.shape

(1, 9, 10)